# IG01 - Inventario

El indicador IG01 verifica los montos de los sobrantes y/o faltantes de inventario. El riesgo que busca mitigar es una posible sub o sobre valoración de los inventarios. El área responsable es la Gerencia de Administración y Finanzas.

El siguiente código es para ejecutar el indicador de Auditoría Continua IG01 - Inventario.

 - ***Fecha creación: 11.08.2020***
 - ***Modificación_1: 13.08.2020***
 - ***Modificación_2: 02.09.2020***

## Importar librerías
---

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Utilidades
---

In [2]:
def clean_blankspace(df):
    #eliminar espacios en nombres de columnas
    columns = list(df.columns)
    columns = [w.replace(' ','') for w in columns]
    df.columns = columns
    #Eliminar espacios en cada celda del dataframe
    for columna in list(df.columns):
        if df.dtypes[columna] == np.dtype('O'):
            df[columna] = df[columna].str.replace(' ','')
    return df

def clean_columns(txt):
    df = txt.drop(columns=txt.columns[0:2]).drop(columns=txt.columns[-1]).dropna(how='all').reset_index(drop=True)
    return df

def num_format(df,col):
    df[col] = df[col].str.replace('.','').str.replace(',','.').astype(float)
    if (col=='DMBTR') or (col=='ERFMG') or (col=='VERPR'):
        df[col] = df[col]*100
    return df

## Cargar tablas
---

In [3]:
T001L_txt = pd.read_csv('T001L.txt', sep='|', header=3, encoding='latin1')
T001K_txt = pd.read_csv('T001K.txt', sep='|', header=3, encoding='latin1')
MSEG_txt = pd.read_csv('MSEG.txt', sep='|', header=3, encoding='latin1')

## Limpieza

#### Eliminar filas y columnas NaN

In [4]:
T001L_df = clean_columns(T001L_txt)
T001K_df = clean_columns(T001K_txt)
MSEG_df = clean_columns(MSEG_txt)

#### Eliminar espacios en nombres de columnas y cada celda

In [5]:
T001L_df = clean_blankspace(T001L_df)
T001K_df = clean_blankspace(T001K_df)
MSEG_df = clean_blankspace(MSEG_df)

#### Cambio de formato a columnas de MSEG

In [6]:
MSEG_df = num_format(MSEG_df,'DMBTR')
MSEG_df = num_format(MSEG_df,'ERFMG')

#### Establecer esquema de datos

In [7]:
schema_T001L = {'WERKS': str,          # Centro 
                'LGORT': str,          # Almacen 
                'LGOBE': str}          # Denominacion

schema_T001K = {'BWKEY': str,          # Centro
                'BUKRS': str}          # Sociedad

schema_MSEG = {'MBLNR': 'float64',     # Doc_material
                'ZEILE': 'float64',    # Posicion
                'BWART': 'float64',    # Clase_mov 
                'MATNR': 'float64',    # Material 
                'WERKS': str,          # Centro 
                'LGORT': str,          # Almacen 
                'CHARG': str,          # Lote 
                'DMBTR': 'float64',    # Monto 
                'ERFMG': 'float64',    # Cantidad 
                'BUDAT_MKPF': str}     # Fecha_contabilidad

T001L_df = T001L_df.astype(schema_T001L)
T001K_df = T001K_df.astype(schema_T001K)
MSEG_df = MSEG_df.astype(schema_MSEG)

#### Cambio de nombre columna BWKEY de T001K a WERKS

In [8]:
T001K_df.rename(columns={'BWKEY':'WERKS'}, inplace = True)

## Ejecución de indicador
---

Primero es necesario realizar cruces entre las tablas para ir incorporando la información de cada una.

### Cruce T001K y T001L

In [9]:
aux_1 = pd.merge(T001K_df, T001L_df, on='WERKS', how='right')
aux_1.head(10)

,WERKS,BUKRS,LGORT,LGOBE
0,A001,A000,BC01,Consumibles
1,A001,A000,BC02,Materialclínico
2,A001,A000,BC03,ActivoFijo
3,A001,A000,BC04,Bajasdeactivo
4,A001,A000,BC05,Marketing
5,A001,A000,BC06,MaterialdePrev
6,A001,A000,BC07,BienesenCustod
7,A002,A000,ACDP,CentralProceso
8,A002,A000,ALMA,AlmacenadoFinal
9,A002,A000,ARMA,ArmadoyEmpacad


### Cruce MSEG y aux_1

In [10]:
aux_2 = pd.merge(MSEG_df, aux_1, on=['WERKS','LGORT'], how='left') #Unimos las dos tablas
aux_2 = aux_2.drop(['MBLNR','ZEILE','MATNR','CHARG','ERFMG','BUDAT_MKPF'], axis=1).rename(columns = {'WERKS':'WERKS'})

aux_2

,BWART,WERKS,LGORT,DMBTR,BUKRS,LGOBE
0,702.0,A002,HTAM,649.0,A000,Ambulatorios
1,702.0,A002,HTAM,307.0,A000,Ambulatorios
2,702.0,A002,HTAM,13868.0,A000,Ambulatorios
3,702.0,A002,HTAM,240.0,A000,Ambulatorios
4,702.0,A002,HTAM,1571.0,A000,Ambulatorios
...,...,...,...,...,...,...
688,701.0,A001,BC02,0.0,A000,Materialclínico
689,701.0,A002,BH01,76199.0,A000,BodegaHospital
690,701.0,A002,BH01,32541.0,A000,BodegaHospital
691,702.0,C001,CEM1,166600.0,C200,CentroMédicoHT


### Calculo de sobrantes y faltantes por almacen y centro

#### Sobrantes

In [11]:
aux_3 = aux_2.loc[(aux_2['BWART']==701),:]
sobrantes = aux_3.groupby(['LGORT','WERKS','BWART']).agg({'DMBTR':'sum'})
sobrantes = sobrantes.rename(columns={'DMBTR':'Sobrantes'})
sobrantes

,,,Sobrantes
LGORT,WERKS,BWART,
BC02,A001,701.0,0.0
BH01,A002,701.0,1040731.0
CEM1,C001,701.0,16566.0
FA01,A002,701.0,11743952.0
HTAM,A002,701.0,35594.0
HTFC,A002,701.0,103351.0
HTFP,A002,701.0,15509.0
HTUR,A002,701.0,2143.0
S100,A100,701.0,1175028.0


#### Faltantes

In [12]:
aux_3 = aux_2.loc[(aux_2['BWART']==702),:]
faltantes = aux_3.groupby(['LGORT','WERKS','BWART']).agg({'DMBTR':'sum'})
faltantes = faltantes.rename(columns={'DMBTR':'Faltantes'})
faltantes

,,,Faltantes
LGORT,WERKS,BWART,
CEM1,C001,702.0,166600.0
FA01,A002,702.0,14840278.0
HTAM,A002,702.0,200986.0
HTFC,A002,702.0,32112.0
HTFP,A002,702.0,36953.0
HTUR,A002,702.0,3975.0
S100,A100,702.0,1164658.0


### Juntar información para crear el indicador

In [13]:
#Quitamos la columna monto de la tabla aux_2 y duplicados que puedan existir

aux_2 = aux_2.drop(['DMBTR','BWART'], axis=1)
aux_2 = aux_2.drop_duplicates()

In [14]:
IG01_df = pd.merge(aux_2, sobrantes, on=['WERKS','LGORT'], how='outer')   #Combinamos tabla aux_2 con la de sobrantes
IG01_df = pd.merge(IG01_df, faltantes, on=['WERKS','LGORT'],how='outer')  #Combinamos la tabla anterior con la de faltantes
IG01_df = IG01_df.fillna(0)                                                           #Rellenamos valores nulos con 0
IG01_df['Ajuste_neto'] = IG01_df['Sobrantes']-IG01_df['Faltantes']                    #Creamos nueva columna con el neto
IG01_df = IG01_df[['BUKRS','WERKS','LGORT','LGOBE','Sobrantes','Faltantes','Ajuste_neto']] #Reordenar columnas
IG01_df = IG01_df.sort_values(by=('LGORT'),ascending=True).reset_index(drop=True)          #Ordenamos por Almacén

IG01_df

,BUKRS,WERKS,LGORT,LGOBE,Sobrantes,Faltantes,Ajuste_neto
0,A000,A001,BC02,Materialclínico,0.0,0.0,0.0
1,A000,A002,BH01,BodegaHospital,1040731.0,0.0,1040731.0
2,C200,C001,CEM1,CentroMédicoHT,16566.0,166600.0,-150034.0
3,A000,A002,FA01,FarmaciaHospita,11743952.0,14840278.0,-3096326.0
4,A000,A002,HTAM,Ambulatorios,35594.0,200986.0,-165392.0
5,A000,A002,HTFC,FarmaciaCAA,103351.0,32112.0,71239.0
6,A000,A002,HTFP,FarmaciaPabello,15509.0,36953.0,-21444.0
7,A000,A002,HTUR,Urgencia,2143.0,3975.0,-1832.0
8,A000,A100,S100,SedeDroguería,1175028.0,1164658.0,10370.0


### Crear tabla resumen por sociedad

**Agrupamos por sociedad**

In [15]:
IG01_aux = IG01_df.drop(['WERKS','LGORT','LGOBE'],axis=1)              #Eliminamos columnas que no se utilizan

IG01_aux_sob = IG01_aux.groupby('BUKRS').agg({'Sobrantes': 'sum'})     #Sumamos los sobrantes por Sociedad
IG01_aux_fal = IG01_aux.groupby('BUKRS').agg({'Faltantes': 'sum'})     #Sumamos los faltantes por Sociedad
IG01_aux_aj = IG01_aux.groupby('BUKRS').agg({'Ajuste_neto': 'sum'})    #Sumamos los ajustes netos por Sociedad

**Unir información de las sumas por sociedad**

In [16]:
IG01_Sociedad = pd.merge(IG01_aux_sob, IG01_aux_fal, on='BUKRS')
IG01_Sociedad = pd.merge(IG01_Sociedad, IG01_aux_aj, on='BUKRS')

IG01_Sociedad.loc['Total']= IG01_Sociedad.sum()

#### Cambio de nombre de columnas

In [17]:
IG01_df.rename(columns={'BUKRS':'Sociedad',
                     'WERKS':'Centro',
                     'LGORT':'Almacen',
                     'LGOBE':'Denominacion'}, 
            inplace = True)

IG01_Sociedad.reset_index().rename(columns={'BUKRS':'Sociedad'}, inplace = True)

### Guardamos el resultado en Excel

In [18]:
nombre_archivo = 'IG01 '+datetime.now().strftime("%d-%m-%y_%Hh%Mm")+'.xlsx'
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')

IG01_df.to_excel(writer, sheet_name='IG01')
IG01_Sociedad.to_excel(writer, sheet_name='IG01_Sociedad')

writer.save()